# import libs

In [1]:
import sys
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

sys.path.append("/home/webphy/Desktop/dnn_processor/")  # just to enable `dataset`
sys.path.append(
    "/home/webphy/Desktop/dnn_processor/dataset/"
)  # just to enable `dataset`

import tensorflow as tf
import optuna
import dataset
import numpy as np
import keras
import joblib
from datetime import datetime
from keras import layers
import keras_cv

Using TensorFlow backend


/home/webphy/Desktop/dnn_processor/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


##  define constants and create dataset


In [2]:
NTRIALS = 800
BATCH_SIZE = 64
EPOCHS = 100

train_images_rgx = "../../dataset/resized_dataset_texturas_v2/*/*.jpeg"
valid_images_rgx = "../../dataset/resized_dataset_texturas_v2_test/*/*.jpeg"

layer_options = ["Conv2D", "Pointwise2D"]
activation_options = ["linear", "relu", "tanh"]
filter_size_options = [3, 4, 5]
num_layer_options = [1, 2, 3]

train_dataset, val_dataset, num_classes = dataset.create_datasets(
    train_images_rgx, valid_images_rgx, BATCH_SIZE
)

## define create model function


In [3]:
def clip_filter_size(fs, ih):
    return min(fs, ih)


def create_layer(
    trial, layer_type, layer_id, max_stride, input_h_size, depth_multiplier
):
    if layer_type == "Conv2D":
        num_filters = trial.suggest_int(
            f"layer_{layer_id}_conv2d_filters", 4, 32, step=4
        )
        filter_size = trial.suggest_categorical(
            f"layer_{layer_id}_conv2d_fsize", filter_size_options
        )

        stride_size = trial.suggest_int(f"layer_{layer_id}_conv2d_ssize", 2, max_stride)
        layer = layers.Conv2D(
            num_filters * depth_multiplier,
            clip_filter_size(filter_size, input_h_size),
            stride_size,
        )

    elif layer_type == "Depthwise2D":
        filter_size = trial.suggest_categorical(
            f"layer_{layer_id}_dwconv2d_fsize", filter_size_options
        )
        stride_size = trial.suggest_int(
            f"layer_{layer_id}_dwconv2d_ssize", 2, max_stride
        )
        layer = layers.DepthwiseConv2D(
            clip_filter_size(filter_size, input_h_size), stride_size
        )

    elif layer_type == "Pointwise2D":
        num_filters = trial.suggest_int(
            f"layer_{layer_id}_pwconv2d_filters", 4, 32, step=4
        )
        stride_size = trial.suggest_int(
            f"layer_{layer_id}_pwconv2d_ssize", 2, max_stride
        )
        layer = layers.Conv2D(num_filters * depth_multiplier, 1, stride_size)

    return layer


def create_augmentation_layers(trial, x):
    # augmix_severity = trial.suggest_float("augmix_severity", 0.0, 0.5)
    brightness_factor = trial.suggest_float("brightness_factor", 0, 0.2)
    gaussian_noise_factor = trial.suggest_float("gaussian_noise_factor", 0, 0.2)
    translate_factor = trial.suggest_float("translate_factor", 0, 0.2)

    # x = keras_cv.layers.AugMix(value_range=(0, 1), severity=augmix_severity)(x)
    x = layers.RandomBrightness(brightness_factor, (0, 1.0))(x)
    x = layers.RandomFlip()(x)
    x = layers.RandomTranslation(translate_factor, translate_factor)(x)
    x = layers.GaussianNoise(gaussian_noise_factor)(x)

    return x


def calibrate_channel(input_shape, s):
    HLS_DENSE_LIMIT = 4000

    x_shape = input_shape[1:]
    h, w, c = x_shape
    h = h // s
    w = w // s

    new_shape = [h, w, c]
    if np.prod(new_shape) > HLS_DENSE_LIMIT:
        # diminui o numero de canais
        while np.prod(new_shape) > HLS_DENSE_LIMIT:
            c -= 1
            new_shape = [h, w, c]
    else:
        # aumenta o numero de canais
        while np.prod(new_shape) < HLS_DENSE_LIMIT:
            c += 1
            new_shape = [h, w, c]
        c -= 1

    if c <= 0:
        c = 1
        new_shape = [None, h, w, c]
        h, w, c, s = calibrate_channel(new_shape, s + 1)
        new_shape = [h, w, c]

    return h, w, c, s


def create_dense_output(trial, x):
    h, w, c, s = calibrate_channel(x.shape, 1)

    # tinsmith layer
    x = layers.Conv2D(c, 1, strides=s)(x)

    # output layer
    x = layers.Flatten()(x)
    x = layers.Dropout(trial.suggest_float("output_dropout", 0.3, 0.6))(x)
    x = layers.Dense(num_classes, activation="softmax")(x)
    return x


def create_global_avg_pooling(trial, x):
    pre_GAP_layer_fsize = trial.suggest_categorical(
        "pre_GAP_layer_fsize", filter_size_options
    )
    pre_GAP_layer_act = trial.suggest_categorical(
        "pre_GAP_layer_act", activation_options
    )
    pre_GAP_layer_fsize = clip_filter_size(pre_GAP_layer_fsize, x.shape[1])
    x = keras.layers.Conv2D(num_classes, pre_GAP_layer_fsize)(x)

    if x.shape[1] > 1:
        x = keras.layers.Activation(pre_GAP_layer_act)(x)
        x = keras.layers.GlobalAvgPool2D()(x)
    else:
        x = keras.layers.Flatten()(x)

    x = keras.layers.Activation("softmax")(x)

    return x


def create_model(trial):
    # add the input layer
    x = x_in = layers.Input(shape=(256, 256, 3))

    x = create_augmentation_layers(trial, x)

    # suggent a number of layer
    num_layers = trial.suggest_categorical("num_layers", num_layer_options)

    # suggest output layer
    output_layer = trial.suggest_categorical("output_layer", ["dense", "pooling"])

    # depth_multiplier for pooling
    depth_multiplier = 1 if output_layer == "dense" else 2

    # limits the stride
    max_stride = 16 // num_layers

    # convolutional layers
    for i in range(num_layers):
        layer_selected = trial.suggest_categorical(f"layer_{i}", layer_options)
        layer = create_layer(
            trial,
            layer_selected,
            i,
            max_stride,
            input_h_size=x.shape[1],
            depth_multiplier=depth_multiplier,
        )
        x = layer(x)
        act_selected = trial.suggest_categorical(f"activation_{i}", activation_options)
        x = layers.Activation(act_selected)(x)

    if output_layer == "dense":
        x = create_dense_output(trial, x)
    else:
        x = create_global_avg_pooling(trial, x)

    return keras.Model(x_in, x)

# if testing

In [4]:
h, w, c, s = calibrate_channel([None, 84,84,20], 1)

class MockTrial:
    def suggest_int(self, name, min, max, step=None, log=False):
        return np.random.randint(min, max)

    def suggest_float(self, name, min, max, step=None, log=False):
        return np.random.uniform(min, max)

    def suggest_categorical(self, name, options):        
        return options[np.random.randint(0, len(options))]

    def suggest_discrete_uniform(self, name, min, max, q):
        return (np.random.randint(min, max) // q) * q

model = create_model(MockTrial())
model.build(input_shape=(BATCH_SIZE, 256, 256, 3))
model.summary()
model.save("saved_models/test/")

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 random_brightness (RandomB  (None, 256, 256, 3)       0         
 rightness)                                                      
                                                                 
 random_flip (RandomFlip)    (None, 256, 256, 3)       0         
                                                                 
 random_translation (Random  (None, 256, 256, 3)       0         
 Translation)                                                    
                                                                 
 gaussian_noise (GaussianNo  (None, 256, 256, 3)       0         
 ise)                                                            
                                                             

INFO:tensorflow:Assets written to: saved_models/test/assets



```
h, w, c, s = calibrate_channel([None, 84,84,20], 1)

class MockTrial:
    def suggest_int(self, name, min, max, step=None, log=False):
        return np.random.randint(min, max)

    def suggest_float(self, name, min, max, step=None, log=False):
        return np.random.uniform(min, max)

    def suggest_categorical(self, name, options):
        print(name)
        return options[np.random.randint(0, len(options) - 1)]

    def suggest_discrete_uniform(self, name, min, max, q):
        return (np.random.randint(min, max) // q) * q

model = create_model(MockTrial())
model.build(input_shape=(BATCH_SIZE, 256, 256, 3))
model.summary()
model.save("saved_models/test/")



```

In [5]:
def create_optimizer(trial):
    lr = trial.suggest_float("lr_inicial", 3e-4, 2e-3, log=True)
    optimizer = keras.optimizers.Adam(lr)
    return optimizer


def create_model_checkpointer(save_path):
    return keras.callbacks.ModelCheckpoint(
        save_path + "weights/",
        "val_accuracy",
        mode="max",
        save_best_only=True,
        save_weights_only=True,
    )


def crate_early_stopper():
    return keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        min_delta=0.1,
        patience=20,
        verbose=0,
        mode="max",
        start_from_epoch=50,
    )

def count_parameters(model):
    total_params = 0
    for layer in model.trainable_variables:
        layer_params = tf.reduce_prod(layer.shape)
        total_params += layer_params
    return total_params.numpy()

def objective(trial):
    BATCH_SIZE = trial.suggest_int("batchsize", 32, 128, step=32)
    
    # load data
    train_dataset, val_dataset, num_classes = dataset.create_datasets(
        train_images_rgx, valid_images_rgx, BATCH_SIZE
    )

    # Build model and optimizer.
    model = create_model(trial)
    assert isinstance(model, keras.Model), "model is not a model"

    optimizer = create_optimizer(trial)

    # compile the model
    model.compile(
        optimizer, loss=keras.losses.categorical_crossentropy, metrics=["accuracy"]
    )

    # crate model checkpointer
    saved_best_model_path = f"saved_models/{EXP_ID}/best_model_t{trial._trial_id}/"
    model.save(saved_best_model_path, True, "tf")  # save all model

    model_chkp = create_model_checkpointer(saved_best_model_path)
    early_stopper = crate_early_stopper()

    # Training and validating cycle.
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=EPOCHS,
        callbacks=[model_chkp, early_stopper],
        verbose="auto",
    )
    
    # load best model
    model.load_weights(saved_best_model_path + "weights/")    

    # Evaluate the model accuracy on the validation set.
    score = model.evaluate(val_dataset, verbose=0)

    # get the model size
    model_size = count_parameters(model)

    # Return best model validation accuracy.
    return score[1], model_size


def search():
    study = optuna.create_study(
        directions=["maximize", "minimize"],
        sampler=optuna.samplers.TPESampler(),
    )
    study.optimize(objective, n_trials=NTRIALS)
    return study

# start search

In [6]:
EXP_ID = "exp7_no_depth_w_GAP_multiObj"
keras.utils.disable_interactive_logging()

# start searching for a model
study = search()

# save the study
now = datetime.now()
joblib.dump(study, "saved_studies/" + now.strftime("%Y:%m:%d:%H:%M:%S") + ".pkl")

[I 2024-02-02 14:23:36,890] Trial 745 finished with values: [0.8199999928474426, 21323.0] and parameters: {'batchsize': 64, 'brightness_factor': 0.1652186594434778, 'gaussian_noise_factor': 0.044762905816837964, 'translate_factor': 0.1743088514422701, 'num_layers': 2, 'output_layer': 'dense', 'layer_0': 'Conv2D', 'layer_0_conv2d_filters': 20, 'layer_0_conv2d_fsize': 4, 'layer_0_conv2d_ssize': 5, 'activation_0': 'linear', 'layer_1': 'Pointwise2D', 'layer_1_pwconv2d_filters': 20, 'layer_1_pwconv2d_ssize': 4, 'activation_1': 'relu', 'output_dropout': 0.4870322743136742, 'lr_inicial': 0.001153047857937543}. 


INFO:tensorflow:Assets written to: saved_models/exp7_no_depth_w_GAP_multiObj/best_model_t746/assets


INFO:tensorflow:Assets written to: saved_models/exp7_no_depth_w_GAP_multiObj/best_model_t746/assets
[I 2024-02-02 14:23:51,511] Trial 746 finished with values: [0.6399999856948853, 4309.0] and parameters: {'batchsize': 96, 'brightness_factor': 0.15620160752338294, 'gaussian_noise_factor': 0.058781691655474116, 'translate_factor': 0.16367919202037293, 'num_layers': 2, 'output_layer': 'pooling', 'layer_0': 'Conv2D', 'layer_0_conv2d_filters': 16, 'layer_0_conv2d_fsize': 5, 'layer_0_conv2d_ssize': 5, 'activation_0': 'linear', 'layer_1': 'Pointwise2D', 'layer_1_pwconv2d_filters': 12, 'layer_1_pwconv2d_ssize': 5, 'activation_1': 'relu', 'pre_GAP_layer_fsize': 3, 'pre_GAP_layer_act': 'relu', 'lr_inicial': 0.00122716944453599}. 


INFO:tensorflow:Assets written to: saved_models/exp7_no_depth_w_GAP_multiObj/best_model_t747/assets


INFO:tensorflow:Assets written to: saved_models/exp7_no_depth_w_GAP_multiObj/best_model_t747/assets
[I 2024-02-02 14:24:06,669] Trial 747 finished with values: [0.7799999713897705, 17582.0] and parameters: {'batchsize': 128, 'brightness_factor': 0.16013854550032539, 'gaussian_noise_factor': 0.049068885161864764, 'translate_factor': 0.15724634115347083, 'num_layers': 2, 'output_layer': 'dense', 'layer_0': 'Conv2D', 'layer_0_conv2d_filters': 8, 'layer_0_conv2d_fsize': 4, 'layer_0_conv2d_ssize': 5, 'activation_0': 'linear', 'layer_1': 'Pointwise2D', 'layer_1_pwconv2d_filters': 20, 'layer_1_pwconv2d_ssize': 2, 'activation_1': 'relu', 'output_dropout': 0.5160028420325419, 'lr_inicial': 0.0012192548305902731}. 


...

INFO:tensorflow:Assets written to: saved_models/exp7_no_depth_w_GAP_multiObj/best_model_t798/assets


INFO:tensorflow:Assets written to: saved_models/exp7_no_depth_w_GAP_multiObj/best_model_t798/assets
[I 2024-02-02 14:37:43,296] Trial 798 finished with values: [0.6399999856948853, 3461.0] and parameters: {'batchsize': 32, 'brightness_factor': 0.16008672788094688, 'gaussian_noise_factor': 0.1418330132349009, 'translate_factor': 0.0766383814323583, 'num_layers': 2, 'output_layer': 'pooling', 'layer_0': 'Conv2D', 'layer_0_conv2d_filters': 4, 'layer_0_conv2d_fsize': 5, 'layer_0_conv2d_ssize': 5, 'activation_0': 'tanh', 'layer_1': 'Pointwise2D', 'layer_1_pwconv2d_filters': 16, 'layer_1_pwconv2d_ssize': 8, 'activation_1': 'relu', 'pre_GAP_layer_fsize': 4, 'pre_GAP_layer_act': 'linear', 'lr_inicial': 0.00031354869484384986}. 


INFO:tensorflow:Assets written to: saved_models/exp7_no_depth_w_GAP_multiObj/best_model_t799/assets


INFO:tensorflow:Assets written to: saved_models/exp7_no_depth_w_GAP_multiObj/best_model_t799/assets
[I 2024-02-02 14:37:58,225] Trial 799 finished with values: [0.8199999928474426, 20219.0] and parameters: {'batchsize': 32, 'brightness_factor': 0.155412149070823, 'gaussian_noise_factor': 0.04793704129341838, 'translate_factor': 0.17506137077874956, 'num_layers': 2, 'output_layer': 'dense', 'layer_0': 'Conv2D', 'layer_0_conv2d_filters': 4, 'layer_0_conv2d_fsize': 4, 'layer_0_conv2d_ssize': 5, 'activation_0': 'linear', 'layer_1': 'Pointwise2D', 'layer_1_pwconv2d_filters': 20, 'layer_1_pwconv2d_ssize': 4, 'activation_1': 'relu', 'output_dropout': 0.3982019807393252, 'lr_inicial': 0.0017107816087214276}. 


['saved_studies/2024:02:02:14:37:58.pkl']

# visualizations

In [12]:
print("Number of finished trials: ", len(study.trials))

trials = study.best_trials
for trial in trials:
    print(f"Best trial id: {trial._trial_id}")
    
    print("  Value: ", trial.values)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

Number of finished trials:  800
Best trial id: 61
  Value:  [0.699999988079071, 757.0]
  Params: 
    batchsize: 32
    brightness_factor: 0.00799756351779848
    gaussian_noise_factor: 0.1102945836448109
    translate_factor: 0.1301138361256372
    num_layers: 1
    output_layer: pooling
    layer_0: Conv2D
    layer_0_conv2d_filters: 4
    layer_0_conv2d_fsize: 4
    layer_0_conv2d_ssize: 5
    activation_0: tanh
    pre_GAP_layer_fsize: 3
    pre_GAP_layer_act: relu
    lr_inicial: 0.00149283809509308
Best trial id: 154
  Value:  [0.6200000047683716, 589.0]
  Params: 
    batchsize: 32
    brightness_factor: 0.06853943597312165
    gaussian_noise_factor: 0.03364490222343767
    translate_factor: 0.16287056293797378
    num_layers: 1
    output_layer: pooling
    layer_0: Conv2D
    layer_0_conv2d_filters: 4
    layer_0_conv2d_fsize: 3
    layer_0_conv2d_ssize: 14
    activation_0: tanh
    pre_GAP_layer_fsize: 3
    pre_GAP_layer_act: relu
    lr_inicial: 0.0015473652021179673
Best 

In [13]:
optuna.visualization.plot_pareto_front(study, target_names=["accuracy", "model_size"])

In [18]:
study_name = "2024:02:02:14:37:58.pkl"
study_2 = joblib.load(
    f"/home/webphy/Desktop/dnn_processor/neural_network/automl/saved_studies/{study_name}"
)

In [19]:
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances

In [ ]:
plot_optimization_history(study_2)

In [ ]:
plot_param_importances(study_2, target_names=["accuracy", "model_size"])

In [24]:
plot_parallel_coordinate(
    study_2,
    [
        "lr_inicial",
        "num_layers",
        "layer_0",
        "activation_0",
    ],
)

ValueError: If the `study` is being used for multi-objective optimization, please specify the `target`.